In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#  *Importing packages*

In [23]:
import numpy as np 
import cv2
from tqdm import tqdm
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
import os

# Check exactly where the videos are
print(os.listdir("/kaggle/input/deep-fake-detection-dfd-entire-original-dataset/DFD_manipulated_sequences/DFD_manipulated_sequences"))

# Dataset paths

In [29]:
REAL_PATH="/kaggle/input/deep-fake-detection-dfd-entire-original-dataset/DFD_original sequences"
FAKE_PATH="/kaggle/input/deep-fake-detection-dfd-entire-original-dataset/DFD_manipulated_sequences/DFD_manipulated_sequences"
OUTPUT_FRAME_SIZE=(128,128)
FRAME_COUNT=10
MAX_VIDEOS=300

In [25]:
def extract_frames(video_path,output_size=(128,128),frame_count=10):
    cap=cv2.VideoCapture(video_path)
    frames=[]
    total_frames=int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    step = max(total_frames // frame_count, 1)  
    for i in range(frame_count):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * step)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, output_size)
        frames.append(frame)
    cap.release()
    return np.array(frames)

In [30]:
data=[]
labels=[]
print("Processing real videos...")
real_videos=os.listdir(REAL_PATH)[:MAX_VIDEOS]
for video_file in tqdm(real_videos):
    video_path=os.path.join(REAL_PATH,video_file)
    frames=extract_frames(video_path,output_size=OUTPUT_FRAME_SIZE,frame_count=FRAME_COUNT)
    if len(frames)==FRAME_COUNT:
        data.append(frames)
        labels.append(0)
print("Processing fake videos")
fake_videos=os.listdir(FAKE_PATH)[:MAX_VIDEOS]
for video_file in tqdm(fake_videos):
    video_path=os.path.join(FAKE_PATH,video_file)
    frames=extract_frames(video_path,output_size=OUTPUT_FRAME_SIZE,frame_count=FRAME_COUNT)
    if len(frames)==FRAME_COUNT:
        data.append(frames)
        labels.append(1)
    

Processing real videos...


100%|██████████| 300/300 [27:50<00:00,  5.57s/it]


Processing fake videos


100%|██████████| 300/300 [27:46<00:00,  5.55s/it]
